In [ ]:
# ============================================================
# Micropillar analysis pipeline (SET4-style)
# Part 1: C1 brightfield -> pillar mask (U-Net inference)
# Part 2: C2 TL-POL -> precipitation quantification (non-pillar only)
# Outputs:
#   outputs/pillar_masks/  (PNG masks)
#   outputs/set4_crystal_metrics.csv
#   outputs/set4_crystal_metrics.xlsx
# ============================================================

# ---------- 0) Install deps ----------
!pip -q install -U segmentation-models-pytorch albumentations opencv-python scikit-image xlsxwriter

# ---------- 1) Imports ----------
from pathlib import Path
import os, re, glob
import urllib.request

import cv2
import numpy as np
import pandas as pd

import torch
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2
from skimage.measure import label, regionprops_table


# ============================================================
# 2) CONFIG  (EDIT THESE)
# ============================================================
ROOT = Path("data/raw_images")  # contains short1/short2/tall1/tall2 (or any nested dirs)

# Where masks will be saved
MASK_OUT_ROOT = Path("outputs/pillar_masks")
MASK_OUT_ROOT.mkdir(parents=True, exist_ok=True)

# Metrics outputs
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)
CSV_OUT  = OUT_DIR / "set4_crystal_metrics.csv"
XLSX_OUT = OUT_DIR / "set4_crystal_metrics.xlsx"

# Model checkpoint location
MODEL_DIR  = Path("models")
MODEL_DIR.mkdir(exist_ok=True)
MODEL_PATH = MODEL_DIR / "unet_pillars_finetuned.pt"

# Box link (must be a *direct* download for urllib to work)
BOX_SHARE_LINK  = "https://cornell.box.com/s/4dyu78bhtpabm98jgz40gdp5wmoe71xn"
DIRECT_DOWNLOAD = BOX_SHARE_LINK + "?download=1"

# Inference settings
DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SHORT_SIDE  = 768
MASK_THR    = 0.47   # your tuned best_thr
OVERWRITE_MASKS = False

# C2 binarization (conservative)
C2_THRESH_MODE = "fixed"   # "fixed" | "percentile" | "otsu+"
C2_FIXED       = 140       # higher = fewer crystals
C2_PERCENTILE  = 94        # if percentile mode
C2_DELTA       = 15        # if otsu+ mode
BLUR_KSIZE     = (3, 3)
MORPH_KERNEL   = (3, 3)

# Imaging calibration (set4)
FIELD_UM   = 1331.2
IMG_PX     = 2048
um_per_px  = FIELD_UM / IMG_PX
mm2_per_px = (um_per_px / 1000.0) ** 2


# ============================================================
# 3) Helpers
# ============================================================
IMG_EXTS = (".png", ".tif", ".tiff", ".jpg", ".jpeg")
def is_img(p: str) -> bool:
    return os.path.splitext(p)[1].lower() in IMG_EXTS

def resize_short(im: np.ndarray, short: int = SHORT_SIDE) -> np.ndarray:
    h, w = im.shape[:2]
    s = short / min(h, w)
    return cv2.resize(im, (int(round(w*s)), int(round(h*s))), interpolation=cv2.INTER_AREA)

def ensure_model_downloaded():
    if MODEL_PATH.exists():
        print("✅ Model exists:", MODEL_PATH)
        return
    print("⬇️ Downloading model checkpoint from Box...")
    try:
        urllib.request.urlretrieve(DIRECT_DOWNLOAD, MODEL_PATH)
        print("✅ Saved:", MODEL_PATH)
    except Exception as e:
        print("❌ Box download failed.")
        print("   Error:", repr(e))
        print("\nFix options:")
        print("  1) Download manually from Box and upload it here, then put it at:")
        print(f"     {MODEL_PATH}")
        print("  2) Replace DIRECT_DOWNLOAD with a true direct file URL.")
        raise

def load_unet_model():
    model = smp.Unet(
        encoder_name="resnet18",
        encoder_weights=None,
        in_channels=3,
        classes=1
    )
    state = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(state)
    model.to(DEVICE).eval()
    return model

def infer_mask_for_c1_path(model, c1_path: str, thr: float = MASK_THR) -> np.ndarray:
    bgr = cv2.imread(c1_path)
    if bgr is None:
        raise FileNotFoundError(f"Unreadable image: {c1_path}")

    H, W = bgr.shape[:2]
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    rs  = resize_short(rgb, short=SHORT_SIDE)

    ten = ToTensorV2()(image=rs)["image"].unsqueeze(0).to(DEVICE).float()
    with torch.no_grad():
        prob = torch.sigmoid(model(ten))[0, 0].cpu().numpy()

    prob_full = cv2.resize(prob, (W, H), interpolation=cv2.INTER_LINEAR)
    mask_bin  = (prob_full > thr).astype(np.uint8) * 255

    k = np.ones((5, 5), np.uint8)
    mask_clean = cv2.morphologyEx(mask_bin, cv2.MORPH_OPEN, k)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, k)

    return mask_clean

def parse_thr_from_name(path: str):
    m = re.search(r"_mask_(\d+(?:\.\d+)?)", os.path.basename(path))
    return float(m.group(1)) if m else None

def pick_best_mask(candidates, thr_target=None):
    if not candidates:
        return None
    if thr_target is not None:
        candidates = sorted(
            candidates,
            key=lambda p: (abs((parse_thr_from_name(p) or 999.0) - thr_target), -os.path.getmtime(p))
        )
    else:
        candidates = sorted(candidates, key=lambda p: -os.path.getmtime(p))
    return candidates[0]

def find_mask_for_c2(c2_path: str, thr_target: float = MASK_THR):
    stem = os.path.splitext(os.path.basename(c2_path))[0]
    base_key = stem.rsplit("_C2", 1)[0]
    rel_path = os.path.relpath(c2_path, ROOT)
    rel_dir  = os.path.dirname(rel_path)

    candidates = glob.glob(str(MASK_OUT_ROOT / rel_dir / f"{base_key}_C1_mask_*.png"))
    mpath = pick_best_mask(candidates, thr_target)
    if mpath:
        return base_key, mpath

    top = rel_dir.split(os.sep)[0] if rel_dir else ""
    if top:
        candidates = glob.glob(str(MASK_OUT_ROOT / top / "**" / f"{base_key}_C1_mask_*.png"), recursive=True)
        mpath = pick_best_mask(candidates, thr_target)
        if mpath:
            return base_key, mpath

    candidates = glob.glob(str(MASK_OUT_ROOT / "**" / f"{base_key}_C1_mask_*.png"), recursive=True)
    mpath = pick_best_mask(candidates, thr_target)
    if mpath:
        return base_key, mpath

    return base_key, None

def to_u8(img: np.ndarray) -> np.ndarray:
    if img.dtype == np.uint8:
        return img
    return cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def binarize_c2(C2_raw: np.ndarray, nonpillar_mask01: np.ndarray) -> np.ndarray:
    img  = to_u8(C2_raw)
    roi  = cv2.bitwise_and(img, img, mask=(nonpillar_mask01 * 255))
    blur = cv2.GaussianBlur(roi, BLUR_KSIZE, 0)

    if C2_THRESH_MODE == "fixed":
        thr_val = int(C2_FIXED)
        _, thr = cv2.threshold(blur, thr_val, 255, cv2.THRESH_BINARY)

    elif C2_THRESH_MODE == "percentile":
        vals = blur[nonpillar_mask01 > 0]
        thr_val = int(np.percentile(vals, C2_PERCENTILE)) if vals.size else 255
        _, thr = cv2.threshold(blur, thr_val, 255, cv2.THRESH_BINARY)

    else:
        t0, _ = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        thr_val = int(max(0, min(255, t0 + C2_DELTA)))
        _, thr = cv2.threshold(blur, thr_val, 255, cv2.THRESH_BINARY)

    thr[nonpillar_mask01 == 0] = 0

    k = np.ones(MORPH_KERNEL, np.uint8)
    thr = cv2.morphologyEx(thr, cv2.MORPH_OPEN,  k, iterations=1)
    thr = cv2.morphologyEx(thr, cv2.MORPH_CLOSE, k, iterations=1)
    return thr

PAT = re.compile(r"^(?P<section>\d+)_(?P<mid>[\d.]+)_(?P<trial>[a-zA-Z0-9]+)_C2\.(png|tif|tiff|jpg|jpeg)$", re.IGNORECASE)

def parse_fields_from_filename(fname: str):
    m = PAT.match(fname)
    if not m:
        return {"section": None, "pillar_diameter_mm": None, "trial": None}
    section = int(m.group("section"))
    mid = m.group("mid")
    trial = str(m.group("trial")).lower()
    diam = float(mid)
    return {"section": section, "pillar_diameter_mm": diam, "trial": trial}

def section_to_porosity(sec):
    if sec is None:
        return None
    return 0.35 if (sec % 2 == 1) else 0.45

def folder_to_form(foldername: str):
    f = foldername.lower()
    if f.startswith("short"):
        return "short"
    if f.startswith("tall"):
        return "tall"
    return ""


# ============================================================
# 4) PART 1 — Infer pillar masks for all C1 images
# ============================================================
ensure_model_downloaded()
model = load_unet_model()

all_imgs = [p for p in glob.glob(str(ROOT / "**" / "*.*"), recursive=True) if is_img(p)]
c1_imgs  = [p for p in all_imgs
            if ("_c1" in os.path.basename(p).lower())
            and ("_answer" not in os.path.basename(p).lower())
            and ("_testanswer" not in os.path.basename(p).lower())]
c1_imgs = sorted(c1_imgs)

print(f"Found {len(c1_imgs)} C1 images under {ROOT}")
if len(c1_imgs) == 0:
    raise FileNotFoundError("❌ No *_C1 images found. Check filenames and ROOT path.")

saved, skipped = 0, 0
for c1_path in c1_imgs:
    rel_path = os.path.relpath(c1_path, ROOT)
    rel_dir  = os.path.dirname(rel_path)
    out_dir  = MASK_OUT_ROOT / rel_dir
    out_dir.mkdir(parents=True, exist_ok=True)

    stem = os.path.splitext(os.path.basename(c1_path))[0]
    out_mask = out_dir / f"{stem}_mask_{MASK_THR:.2f}.png"

    if out_mask.exists() and not OVERWRITE_MASKS:
        skipped += 1
        continue

    mask = infer_mask_for_c1_path(model, c1_path, thr=MASK_THR)
    cv2.imwrite(str(out_mask), mask)
    saved += 1

print(f"✅ Masks complete: saved={saved}, skipped_existing={skipped}")
print("Mask root:", MASK_OUT_ROOT)


# ============================================================
# 5) PART 2 — Quantify crystals in all C2 images using the masks
# ============================================================
c2_imgs = [p for p in glob.glob(str(ROOT / "**" / "*.*"), recursive=True)
           if is_img(p)
           and ("_c2" in os.path.basename(p).lower())
           and ("_answer" not in os.path.basename(p).lower())]
c2_imgs = sorted(c2_imgs)

print(f"Found {len(c2_imgs)} C2 images under {ROOT}")
if len(c2_imgs) == 0:
    raise FileNotFoundError("❌ No *_C2 images found. Check filenames and ROOT path.")

rows = []
missing_masks = 0

for c2_path in c2_imgs:
    base_key, mask_path = find_mask_for_c2(c2_path, thr_target=MASK_THR)
    if not mask_path:
        missing_masks += 1
        continue

    C2 = cv2.imread(c2_path, cv2.IMREAD_GRAYSCALE)
    M  = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if C2 is None or M is None:
        continue

    if M.shape != C2.shape:
        M = cv2.resize(M, (C2.shape[1], C2.shape[0]), interpolation=cv2.INTER_NEAREST)

    total_px     = M.size
    pillar_px    = int((M > 0).sum())
    nonpillar_px = total_px - pillar_px

    nonpillar01  = (M == 0).astype(np.uint8)
    precip255    = binarize_c2(C2, nonpillar01)
    crystal_px   = int((precip255 > 0).sum())

    pct_pillar            = 100.0 * pillar_px / total_px
    pct_nonpillar         = 100.0 * nonpillar_px / total_px
    pct_crystal_total     = 100.0 * crystal_px / total_px
    pct_crystal_nonpillar = 100.0 * crystal_px / max(1, nonpillar_px)

    lab = label(precip255 > 0, connectivity=2)
    props = regionprops_table(lab, properties=("area",))
    areas_px = np.array(props["area"], dtype=float) if len(props["area"]) else np.array([])
    n = int(len(areas_px))
    mean_px = float(areas_px.mean()) if n > 0 else 0.0
    std_px  = float(areas_px.std(ddof=1)) if n > 1 else 0.0
    tot_area_px = float(areas_px.sum()) if n > 0 else 0.0

    fname = os.path.basename(c2_path)
    fields = parse_fields_from_filename(fname)

    rel_dir = os.path.dirname(os.path.relpath(c2_path, ROOT))
    top_folder = rel_dir.split(os.sep)[0] if rel_dir else ""
    form = folder_to_form(top_folder)
    porosity = section_to_porosity(fields["section"])

    rows.append({
        "file": fname,
        "rel_dir": rel_dir,
        "top_folder": top_folder,
        "form": form,
        "base": base_key,
        "mask_file": os.path.basename(mask_path),
        "mask_thr": MASK_THR,

        "section": fields["section"],
        "pillar_diameter_mm": fields["pillar_diameter_mm"],
        "trial": fields["trial"],
        "porosity": porosity,

        "n_crystals": n,
        "mean_area_px": mean_px,
        "std_area_px": std_px,
        "total_area_px": tot_area_px,

        "mean_area_mm2": mean_px * mm2_per_px,
        "std_area_mm2": std_px * mm2_per_px,
        "total_area_mm2": tot_area_px * mm2_per_px,

        "pillar_area_px": pillar_px,
        "nonpillar_area_px": nonpillar_px,
        "percent_pillar_area": pct_pillar,
        "percent_nonpillar_area": pct_nonpillar,
        "percent_crystal_total": pct_crystal_total,
        "percent_crystal_in_nonpillar": pct_crystal_nonpillar,
    })

df = pd.DataFrame(rows)
print(f"✅ Quantified {len(df)} images. Missing masks for {missing_masks} C2 images.")

df.to_csv(CSV_OUT, index=False)
print("✅ Saved CSV:", CSV_OUT)

with pd.ExcelWriter(XLSX_OUT, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="metrics", index=False)

print("📗 Saved Excel:", XLSX_OUT)
display(df.head(10))
